# App Workflow
This notebook will outline the workflow needed for the Shiny app

In [1]:
# import packages
import pandas as pd
import geopandas as gpd
import pickle
from datetime import datetime
import json

# visulaztion packages
import ipyleaflet
from branca.colormap import linear
import ipywidgets

# import functions
import sys
sys.path.append('/Users/justinwilliams/projects/gun-violence/src')
from shiny_functions import groupby_mult, rate_per_1k, rgb_to_hex, divide_metric, int_choro_ipyleaflet

# supress warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# import df
df_gun = pd.read_pickle("./data/pickle/df_gun")
df_census = pd.read_pickle("./data/pickle/df_us_census")
df_gun_laws = pd.read_pickle("./data/pickle/df_gun_laws")
us_shp = gpd.GeoDataFrame(pd.read_pickle("./data/pickle/us_shp"))

In [3]:
# drop dc
df_gun=df_gun[df_gun["state"] != "District of Columbia"]

In [4]:
# create total injured/killed column per month
df_gun["total_injured_killed"] = \
df_gun["#_killed"] + df_gun["#_injured"]

In [5]:
# create variables for groupby func
groupby_list = ["year", "monthname","state"]
agg_dict = {"#_injured":"sum","#_killed":"sum", "total_injured_killed":"sum",
            "incident_id":"count","month":"first",}

sort_order = ["incident_id","#_injured","#_killed"]

# apply function
df_yr_mon_state_grp = groupby_mult(df_gun, groupby_list, agg_dict, sort_order)

# preview results
df_yr_mon_state_grp.head(12)

,year,monthname,state,#_injured,#_killed,total_injured_killed,incident_id,month
0,2021,Jul,Illinois,79,8,87,17,7
1,2020,Jul,Illinois,82,15,97,15,7
2,2022,Jul,Illinois,79,14,93,14,7
3,2021,Jun,Illinois,66,11,77,13,6
4,2021,Aug,Illinois,52,9,61,13,8
5,2020,Jun,Illinois,50,5,55,13,6
6,2021,Sep,Illinois,45,9,54,11,9
7,2020,Jun,New York,50,3,53,10,6
8,2020,Aug,Illinois,41,5,46,10,8
9,2021,Jun,Ohio,44,6,50,9,6


In [6]:
# rename incident id column to count
df_yr_mon_state_grp.rename(columns={"incident_id":"count"}, inplace=True)

Create date column as datetime from `year` and `monthname` for date range in ui portion of shiny app

In [7]:
df_yr_mon_state_grp["date"] = pd.to_datetime(df_yr_mon_state_grp[['year', 'month']]\
                                             .assign(DAY=1)).astype('str')

df_yr_mon_state_grp["date"]= \
df_yr_mon_state_grp["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").date())

In [8]:
df_yr_mon_state_grp["date"].dtype

dtype('O')

In [9]:
max(df_yr_mon_state_grp["date"])

datetime.date(2022, 12, 1)

In [10]:
df_yr_mon_state_grp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839 entries, 0 to 838
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   year                  839 non-null    int64 
 1   monthname             839 non-null    object
 2   state                 839 non-null    object
 3   #_injured             839 non-null    int64 
 4   #_killed              839 non-null    int64 
 5   total_injured_killed  839 non-null    int64 
 6   count                 839 non-null    int64 
 7   month                 839 non-null    int64 
 8   date                  839 non-null    object
dtypes: int64(6), object(3)
memory usage: 59.1+ KB


Pickle df for shiny app.

In [11]:
df_yr_mon_state_grp.to_pickle("./data/pickle/df_yr_mon_state")

Subset for year THEN join

In [12]:
# define start and enddate
startdate = pd.to_datetime("2019-01-01").date()
enddate = pd.to_datetime("2022-12-01").date()

In [13]:
# subset for one year, and month otherwise won't be able to graph in this manner
df_subset=df_yr_mon_state_grp[(df_yr_mon_state_grp["date"] >= startdate) &\
                     (df_yr_mon_state_grp["date"] < enddate)]

Group subset by state

In [14]:
# create variables for function
groupby_list = ["state"]
agg_dict = {"count":"sum","#_injured":"sum","#_killed":"sum","total_injured_killed":"sum"}
sort_list = ["count","#_injured","#_killed"]

# apply function
df_subset_state_grp = groupby_mult(df_yr_mon_state_grp, groupby_list, agg_dict, sort_list)

# preview result
df_subset_state_grp.head()

,state,count,#_injured,#_killed,total_injured_killed
0,Illinois,214,963,153,1116
1,Texas,146,584,200,784
2,California,145,559,179,738
3,New York,104,455,57,512
4,Pennsylvania,104,432,66,498


In [15]:
# join census data for population
df_subset_state_census=df_subset_state_grp.merge(df_census,how="outer")

Get ratio per 1k by state

In [16]:
# create lists for function
col_list = ["count","#_injured", "#_killed", "total_injured_killed"]
new_col_list = ["count_per_1k", "injured_per_1k","killed_per_1k","total_per_1k"]

# pass to function
rate_per_1k(df_subset_state_census, col_list, new_col_list)

# preview
df_subset_state_census.head()

,state,count,#_injured,#_killed,total_injured_killed,population,state_fips,pop_per_1k,count_per_1k,injured_per_1k,killed_per_1k,total_per_1k
0,Illinois,214.0,963.0,153.0,1116.0,12716164.0,17,12716.0,1.68,7.57,1.20,8.78
1,Texas,146.0,584.0,200.0,784.0,28635442.0,48,28635.0,0.51,2.04,0.70,2.74
2,California,145.0,559.0,179.0,738.0,39346023.0,06,39346.0,0.37,1.42,0.45,1.88
3,New York,104.0,455.0,57.0,512.0,19514849.0,36,19515.0,0.53,2.33,0.29,2.62
4,Pennsylvania,104.0,432.0,66.0,498.0,12794885.0,42,12795.0,0.81,3.38,0.52,3.89


Define metric division

In [45]:
# create division of metric for legend
metric=divide_metric(df_subset_state_census,"killed_per_1k")
metric

[0.0, 0.31, 0.61, 0.92, 1.23, 1.53, 1.84]

Define rgb list

In [44]:
rgb_list = rgb_to_hex(linear.Blues_07.colors)

Then we need to join gun laws

In [35]:
# merge gun laws
df_subset_census_laws = df_subset_state_census.merge(df_gun_laws)

Now we need to join the shapefile

In [36]:
df_subset_census_laws.head()

,state,count,#_injured,#_killed,total_injured_killed,population,state_fips,pop_per_1k,count_per_1k,injured_per_1k,killed_per_1k,total_per_1k,lawtotal
0,Illinois,214.0,963.0,153.0,1116.0,12716164.0,17,12716.0,1.68,7.57,1.20,8.78,70
1,Texas,146.0,584.0,200.0,784.0,28635442.0,48,28635.0,0.51,2.04,0.70,2.74,18
2,California,145.0,559.0,179.0,738.0,39346023.0,06,39346.0,0.37,1.42,0.45,1.88,111
3,New York,104.0,455.0,57.0,512.0,19514849.0,36,19515.0,0.53,2.33,0.29,2.62,77
4,Pennsylvania,104.0,432.0,66.0,498.0,12794885.0,42,12795.0,0.81,3.38,0.52,3.89,39


In [37]:
us_shp.head()

,statefp,statens,affgeoid,geoid,stusps,name,lsad,aland,awater,geometry
0,56,01779807,0400000US56,56,WY,Wyoming,00,251458712294,1867503716,"POLYGON ((-111.05456 45.00095, -111.04507 45.0..."
1,02,01785533,0400000US02,02,AK,Alaska,00,1478942847588,245378425142,"MULTIPOLYGON (((179.48246 51.98284, 179.48657 ..."
2,24,01714934,0400000US24,24,MD,Maryland,00,25151992308,6979074857,"MULTIPOLYGON (((-76.05015 37.98691, -76.04998 ..."
4,05,00068085,0400000US05,05,AR,Arkansas,00,134660767709,3121950081,"POLYGON ((-94.61792 36.49941, -94.61765 36.499..."
5,38,01779797,0400000US38,38,ND,North Dakota,00,178694310772,4414779956,"POLYGON ((-104.04868 48.86378, -104.04865 48.8..."


In [38]:
# join shapefile with year, month, state agg
gdf_subset = gpd.GeoDataFrame(df_subset_census_laws.merge(us_shp, left_on="state", right_on="name"))

In [39]:
# reset index
gdf_subset.set_index("state_fips", inplace=True)

In [40]:
# make gdf geojson
geojson_gdf_subset=gdf_subset.to_json()
geojson_gdf_subset=json.loads(geojson_gdf_subset)

In [42]:
# use function to plot
int_choro_ipyleaflet(geojson_gdf_subset, colormap=linear.Blues_07,gdf=gdf_subset, 
                     value="killed_per_1k",title="Killed per 1k", 
                     metric=metric, rgb_list=rgb_list)